# USGS Site Coverage Analysis

This notebook validates assumptions about USGS data availability for the Missouri River Basin (HUC 10).

**Key findings:**
- ~13,000 total stream gage sites exist in USGS database for HUC 10
- ~25-30% of sites have discharge (water level) data
- Of those with discharge data, ~1/3 have instantaneous values (IV, 15-min resolution)

**Data Source:** USGS OGC API (https://api.waterdata.usgs.gov/ogcapi/v0/)

In [7]:
import requests
import pandas as pd

# Configuration
HUC_REGION = "10"  # Missouri River Basin
BASE_URL = "https://api.waterdata.usgs.gov/ogcapi/v0"

# States that overlap with Missouri Basin
MISSOURI_BASIN_STATES = [
    'Montana', 'Wyoming', 'North Dakota', 'South Dakota',
    'Nebraska', 'Colorado', 'Kansas', 'Missouri', 'Iowa', 'Minnesota'
]

## 1. Total Stream Gage Sites in HUC 10

Query USGS for ALL stream sites (site_type_code=ST) in the Missouri Basin states,
then filter to those within HUC region 10.

In [8]:
def get_stream_sites_by_state(state_name: str) -> list[dict]:
    """Query all stream sites in a state from USGS OGC API."""
    url = f"{BASE_URL}/collections/monitoring-locations/items"
    params = {
        'f': 'json',
        'limit': 15000,
        'site_type_code': 'ST',
        'state_name': state_name
    }
    resp = requests.get(url, params=params, timeout=120)
    resp.raise_for_status()
    return resp.json().get('features', [])

# Collect all stream sites in Missouri Basin states
all_sites = []
state_counts = {}

print("Querying stream sites per state...")
for state in MISSOURI_BASIN_STATES:
    sites = get_stream_sites_by_state(state)
    huc10_sites = [s for s in sites 
                   if (s.get('properties', {}).get('hydrologic_unit_code') or '').startswith(HUC_REGION)]
    state_counts[state] = {'total': len(sites), 'huc10': len(huc10_sites)}
    all_sites.extend(huc10_sites)
    print(f"  {state}: {len(sites):,} total, {len(huc10_sites):,} in HUC {HUC_REGION}")

total_stream_sites = len(all_sites)
print(f"\nTotal stream gage sites in HUC {HUC_REGION}: {total_stream_sites:,}")

Querying stream sites per state...
  Montana: 3,646 total, 2,805 in HUC 10
  Wyoming: 2,569 total, 1,607 in HUC 10
  North Dakota: 1,253 total, 715 in HUC 10
  South Dakota: 2,368 total, 2,280 in HUC 10
  Nebraska: 1,725 total, 1,716 in HUC 10
  Colorado: 6,099 total, 1,195 in HUC 10
  Kansas: 1,751 total, 905 in HUC 10
  Missouri: 1,958 total, 949 in HUC 10
  Iowa: 1,581 total, 395 in HUC 10
  Minnesota: 2,571 total, 54 in HUC 10

Total stream gage sites in HUC 10: 12,621


## 2. Sites with Discharge Data

Query the time-series-metadata collection to find sites that have discharge
(parameter code 00060) data available.

In [9]:
def get_discharge_timeseries_by_state(state_name: str) -> list[dict]:
    """Query time series metadata for discharge data in a state."""
    url = f"{BASE_URL}/collections/time-series-metadata/items"
    params = {
        'f': 'json',
        'limit': 50000,
        'parameter_code': '00060',  # Discharge
        'state_name': state_name
    }
    resp = requests.get(url, params=params, timeout=120)
    resp.raise_for_status()
    return resp.json().get('features', [])

# Collect discharge data availability
all_discharge_sites = set()
all_iv_sites = set()
all_daily_sites = set()

print("Querying discharge time series metadata per state...")
for state in MISSOURI_BASIN_STATES:
    ts_records = get_discharge_timeseries_by_state(state)
    
    state_discharge = set()
    state_iv = set()
    state_daily = set()
    
    for record in ts_records:
        props = record.get('properties', {})
        huc = props.get('hydrologic_unit_code') or ''
        
        # Only count sites in HUC 10
        if not huc.startswith(HUC_REGION):
            continue
        
        site_id = props.get('monitoring_location_id', '')
        if not site_id:
            continue
        
        state_discharge.add(site_id)
        
        # Classify by data type
        comp_id = props.get('computation_identifier', '')
        period_id = props.get('computation_period_identifier', '')
        
        if comp_id == 'Instantaneous' or period_id == 'Points':
            state_iv.add(site_id)
        if period_id == 'Daily':
            state_daily.add(site_id)
    
    all_discharge_sites.update(state_discharge)
    all_iv_sites.update(state_iv)
    all_daily_sites.update(state_daily)
    
    print(f"  {state}: {len(state_discharge):,} discharge, {len(state_iv):,} IV, {len(state_daily):,} daily")

sites_with_discharge = len(all_discharge_sites)
sites_with_iv = len(all_iv_sites)
sites_with_daily = len(all_daily_sites)

print(f"\nSites with discharge data: {sites_with_discharge:,}")

Querying discharge time series metadata per state...
  Montana: 941 discharge, 241 IV, 655 daily
  Wyoming: 649 discharge, 179 IV, 501 daily
  North Dakota: 202 discharge, 64 IV, 135 daily
  South Dakota: 480 discharge, 168 IV, 257 daily
  Nebraska: 504 discharge, 196 IV, 354 daily
  Colorado: 360 discharge, 139 IV, 283 daily
  Kansas: 288 discharge, 132 IV, 210 daily
  Missouri: 296 discharge, 151 IV, 194 daily
  Iowa: 120 discharge, 41 IV, 64 daily
  Minnesota: 20 discharge, 0 IV, 3 daily

Sites with discharge data: 3,860


## 3. Coverage Analysis

In [10]:
# Calculate percentages
discharge_pct = (sites_with_discharge / total_stream_sites) * 100
iv_pct_of_discharge = (sites_with_iv / sites_with_discharge) * 100
iv_pct_of_total = (sites_with_iv / total_stream_sites) * 100
daily_pct_of_discharge = (sites_with_daily / sites_with_discharge) * 100

print("Coverage Analysis:")
print(f"  Discharge sites as % of all stream sites: {discharge_pct:.1f}%")
print(f"  IV sites as % of discharge sites: {iv_pct_of_discharge:.1f}%")
print(f"  Daily sites as % of discharge sites: {daily_pct_of_discharge:.1f}%")

Coverage Analysis:
  Discharge sites as % of all stream sites: 30.6%
  IV sites as % of discharge sites: 34.0%
  Daily sites as % of discharge sites: 68.8%


## 4. Summary Table

In [11]:
summary = pd.DataFrame({
    "Category": [
        "All stream sites in HUC 10",
        "Sites with discharge data",
        "Sites with IV data (15-min)",
        "Sites with daily data"
    ],
    "Count": [
        total_stream_sites,
        sites_with_discharge,
        sites_with_iv,
        sites_with_daily
    ],
    "% of Total": [
        "100%",
        f"{discharge_pct:.1f}%",
        f"{iv_pct_of_total:.1f}%",
        f"{(sites_with_daily / total_stream_sites) * 100:.1f}%"
    ],
    "% of Discharge Sites": [
        "-",
        "100%",
        f"{iv_pct_of_discharge:.1f}%",
        f"{daily_pct_of_discharge:.1f}%"
    ]
})

summary

,Category,Count,% of Total,% of Discharge Sites
0,All stream sites in HUC 10,12621,100%,-
1,Sites with discharge data,3860,30.6%,100%
2,Sites with IV data (15-min),1311,10.4%,34.0%
3,Sites with daily data,2656,21.0%,68.8%


## 5. Visualization

In [12]:
def text_bar(count, max_count, width=50):
    """Create a text-based progress bar."""
    filled = int((count / max_count) * width)
    return chr(9608) * filled + chr(9617) * (width - filled)

print(f"USGS Site Coverage Funnel - Missouri Basin (HUC {HUC_REGION})")
print("=" * 75)
print()
print(f"All Stream Sites     {text_bar(total_stream_sites, total_stream_sites)} {total_stream_sites:>6,} (100%)")
print(f"With Discharge Data  {text_bar(sites_with_discharge, total_stream_sites)} {sites_with_discharge:>6,} ({discharge_pct:.1f}%)")
print(f"With IV Data         {text_bar(sites_with_iv, total_stream_sites)} {sites_with_iv:>6,} ({iv_pct_of_total:.1f}%)")
print()

# Optional matplotlib visualization
try:
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    categories = [
        f"All Stream Sites\n({total_stream_sites:,})",
        f"With Discharge\n({sites_with_discharge:,})",
        f"With IV Data\n({sites_with_iv:,})"
    ]
    counts = [total_stream_sites, sites_with_discharge, sites_with_iv]
    colors = ["#3498db", "#2ecc71", "#e74c3c"]
    
    bars = ax.barh(categories, counts, color=colors)
    ax.set_xlabel("Number of Sites")
    ax.set_title(f"USGS Site Coverage Funnel - Missouri Basin (HUC {HUC_REGION})")
    
    for bar, count in zip(bars, counts):
        pct = (count / total_stream_sites) * 100
        ax.text(count + 200, bar.get_y() + bar.get_height()/2, 
                f"{pct:.1f}%", va="center", fontsize=10)
    
    plt.tight_layout()
    plt.show()
except ImportError:
    print("(Install matplotlib for graphical visualization: uv add matplotlib)")

USGS Site Coverage Funnel - Missouri Basin (HUC 10)

All Stream Sites     ██████████████████████████████████████████████████ 12,621 (100%)
With Discharge Data  ███████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  3,860 (30.6%)
With IV Data         █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  1,311 (10.4%)

(Install matplotlib for graphical visualization: uv add matplotlib)


## 6. Conclusions

The analysis confirms the README assertions:

1. **~13,000 total stream gage sites** exist in the USGS database for HUC 10 (Missouri Basin)
2. **~25-30% have discharge data** - Only about a quarter to a third of sites have actual water level measurements
3. **~1/3 of discharge sites have IV data** - High-resolution (15-min) data is available for roughly a third of the sites with any discharge data

### Implications for Model Training

- We can only train on sites with discharge data (~3,400-3,800 sites)
- For high-resolution forecasting, we're limited to ~1,300 IV sites
- Daily models can use ~2,600-2,700 sites